### 환경설정

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get MLflow tracking URI from environment variable
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
print(mlflow_tracking_uri)

In [ ]:
import mlflow

# Set MLflow tracking URI
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [ ]:
# Create or set MLflow Experiment
experiment_name = "iris_classification"
mlflow.set_experiment(experiment_name)

### 데이터 불러오기

In [ ]:
from sklearn.datasets import load_iris

# Load dataset
iris = load_iris()
X = iris.data
y = iris.target

### 훈련, 테스트 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 데이터 전처리

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Define PyTorch dataset and dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

### 모델 정의

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(4, 8)
        self.fc2 = nn.Linear(8, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
model = NeuralNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 훈련

In [ ]:
# Start MLflow run
with mlflow.start_run(run_name="NeuralNetwork"):
    # Log parameters
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("learning_rate", 0.001)

    # Train model
    for epoch in range(100):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Log loss
        mlflow.log_metric("loss", loss.item(), step=epoch)

    # Evaluate model on test set
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = torch.sum(predicted == y_test_tensor).item() / len(y_test_tensor)

    # Log accuracy
    mlflow.log_metric("accuracy", accuracy)

    # Save model
    mlflow.pytorch.log_model(model, "model")